In [1]:
"""
We use following lines because we are running on Google Colab
If you are running notebook on a local computer, you don't need these
"""
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow2/knowledge_graph_completion/wn18/main')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!pip install tensorflow-gpu==2.0.0-alpha0 

In [3]:
import tensorflow as tf
import pprint
import logging
import time

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())

TensorFlow Version 2.0.0-alpha0
GPU Enabled: True


In [0]:
def get_vocab(f_path):
  word2idx = {}
  with open(f_path) as f:
    for i, line in enumerate(f):
      line = line.rstrip()
      word2idx[line] = i
  return word2idx

In [0]:
"""
we use 1vN fast evaluation as purposed in ConvE paper:
"https://arxiv.org/abs/1707.01476"
sp2o is a dictionary that maps a pair of <subject, predicate>
to multiple possible corresponding <objects> in graph
"""
def make_sp2o(f_paths, e2idx, r2idx):
    sp2o = {}
    for f_path in f_paths:
      with open(f_path) as f:
        for line in f:
            line = line.rstrip()
            s, p, o = line.split()
            s, p, o = e2idx[s], r2idx[p], e2idx[o]
            if (s,p) not in sp2o:
                sp2o[(s,p)] = [o]
            else:
                if o not in sp2o[(s,p)]:
                    sp2o[(s,p)].append(o)
    return sp2o

In [0]:
def map_fn(x, y):
  i, v, s = y[0]
  one_hot = tf.SparseTensor(i, v, s)
  return x, (one_hot, y[1], y[2])


# stream data from text files
def data_generator(f_path, params, sp2o):
  with open(f_path) as f:
    print('Reading', f_path)
    for line in f:
      line = line.rstrip()
      s, p, o = line.split()
      s, p, o = params['e2idx'][s], params['r2idx'][p], params['e2idx'][o]
      sparse_i = [[x] for x in sp2o[(s, p)]]
      sparse_v = [1.] * len(sparse_i)
      sparse_s = [len(params['e2idx'])]
      yield ((s, p), ((sparse_i, sparse_v, sparse_s), o, len(sparse_i)))


def dataset(is_training, params, sp2o):
  _shapes = (([], []), (([None, 1], [None], [1]), [], []))
  _types = ((tf.int32, tf.int32),
            ((tf.int64, tf.float32, tf.int64), tf.int32, tf.int32))
  
  if is_training:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['train_path'], params, sp2o),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.shuffle(params['num_samples'])
    ds = ds.map(map_fn)
    ds = ds.batch(params['batch_size'])
  
  else:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['test_path'], params, sp2o),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.map(map_fn)
    ds = ds.batch(params['batch_size'])
  
  return ds

In [0]:
def update_metrics(scores, query, metrics):
  to_float = lambda x: tf.cast(x, tf.float32)
  
  _, i = tf.math.top_k(scores, sorted=True, k=scores.shape[1])
  query = tf.expand_dims(query, 1)
  is_query = to_float(tf.equal(i, query))
  r = tf.argmax(is_query, -1) + 1
  
  mrr = 1. / to_float(r)
  hits_10 = to_float(tf.less_equal(r, 10))
  hits_3 = to_float(tf.less_equal(r, 3))
  hits_1 = to_float(tf.less_equal(r, 1))
  
  metrics['mrr'].update_state(mrr)
  metrics['hits_10'].update_state(hits_10)
  metrics['hits_3'].update_state(hits_3)
  metrics['hits_1'].update_state(hits_1)

In [0]:
class DistMult(tf.keras.Model):
  def __init__(self, params):
    super().__init__()
    self.embed_ent = tf.keras.layers.Embedding(input_dim=len(params['e2idx']),
                                               output_dim=params['embed_dim'],
                                               embeddings_initializer=tf.initializers.RandomUniform(),
                                               name='Entity')
    self.embed_rel = tf.keras.layers.Embedding(input_dim=len(params['r2idx']),
                                               output_dim=params['embed_dim'],
                                               embeddings_initializer=tf.initializers.RandomUniform(),
                                               name='Relation')
  
  def call(self, inputs):
    s, p = inputs
    s = self.embed_ent(s)
    p = self.embed_rel(p)
    x = tf.matmul(s * p, self.embed_ent.embeddings, transpose_b=True)
    return x

In [0]:
params = {
    'train_path': '../data/wn18/train.txt',
    'valid_path': '../data/wn18/valid.txt',
    'test_path': '../data/wn18/test.txt',
    'entity_path': '../vocab/entity.txt',
    'relation_path': '../vocab/relation.txt',
    'batch_size': 128,
    'embed_dim': 200,
    'num_samples': 141442,
    'lr': 3e-3,
    'num_patience': 3,
}

In [0]:
params['e2idx']  = get_vocab(params['entity_path'])
params['r2idx'] = get_vocab(params['relation_path'])
sp2o_tr = make_sp2o([params['train_path']], params['e2idx'], params['r2idx'])
sp2o_all = make_sp2o([params['train_path'],
                      params['test_path'],
                      params['valid_path']], params['e2idx'], params['r2idx'])

In [0]:
def is_descending(history: list):
  history = history[-(params['num_patience']+1):]
  for i in range(1, len(history)):
    if history[i-1] <= history[i]:
      return False
  return True 

In [12]:
model = DistMult(params)
model.build(input_shape=[[None], [None]])
pprint.pprint([(v.name, v.shape) for v in model.trainable_variables])

decay_lr = tf.optimizers.schedules.ExponentialDecay(params['lr'], 1000, 0.96)
optim = tf.optimizers.Adam(params['lr'])
global_step = 0

best_mrr = 0.
history_mrr = []

t0 = time.time()
logger = logging.getLogger('tensorflow')
logger.setLevel(logging.INFO)


while True:
  # TRAINING
  for ((s, p), (multi_o, o, num_pos)) in dataset(is_training=True, params=params, sp2o=sp2o_tr):
    with tf.GradientTape() as tape:
      logits = model((s, p))
      multi_o = tf.sparse.to_dense(multi_o, validate_indices=False)
      num_neg = len(params['e2idx']) - num_pos
      pos_weight = tf.expand_dims(tf.cast(num_neg/num_pos, tf.float32), 1)
      loss = tf.nn.weighted_cross_entropy_with_logits(targets=multi_o, logits=logits, pos_weight=pos_weight)
      loss = tf.reduce_mean(loss)
    
    optim.lr.assign(decay_lr(global_step))
    grads = tape.gradient(loss, model.trainable_variables)
    optim.apply_gradients(zip(grads, model.trainable_variables))

    if global_step % 50 == 0:
      logger.info("Step {} | Loss: {:.4f} | Spent: {:.1f} secs | LR: {:.6f}".format(
          global_step, loss.numpy().item(), time.time()-t0, optim.lr.numpy().item()))
      t0 = time.time()
    global_step += 1
  
  # EVALUATION
  metrics = {
    'mrr': tf.metrics.Mean(),
    'hits_10': tf.metrics.Mean(),
    'hits_3': tf.metrics.Mean(),
    'hits_1': tf.metrics.Mean(),
  }
  for ((s, p), (multi_o, o, num_pos)) in dataset(is_training=False, params=params, sp2o=sp2o_all):
    logits = model((s, p))
    multi_o = tf.sparse.to_dense(multi_o, validate_indices=False)
    # create masks for Filtered MRR
    o_one_hot = tf.one_hot(o, len(params['e2idx']))
    unwanted = multi_o - o_one_hot
    masks = tf.cast(tf.equal(unwanted, 0.), tf.float32)
    scores = tf.sigmoid(logits) * masks
    
    update_metrics(scores=scores, query=o, metrics=metrics)
  
  logger.info("MRR: {:.3f}| Hits@10: {:.3f} | Hits@3: {:.3f} | Hits@1: {:.3f}".format(
    metrics['mrr'].result().numpy(),
    metrics['hits_10'].result().numpy(),
    metrics['hits_3'].result().numpy(),
    metrics['hits_1'].result().numpy()))
  
  mrr = metrics['mrr'].result().numpy()
  history_mrr.append(mrr)
  
  if mrr > best_mrr:
    best_mrr = mrr
    # you can save model here
  logger.info("Best MRR: {:.3f}".format(best_mrr))
  
  if len(history_mrr) > params['num_patience'] and is_descending(history_mrr):
    logger.info("MRR not improved over {} epochs, Early Stop".format(params['num_patience']))
    break

W0419 02:45:29.234123 140363059820416 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:410: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


[('Entity/embeddings:0', TensorShape([40943, 200])),
 ('Relation/embeddings:0', TensorShape([18, 200]))]
Reading ../data/wn18/train.txt


I0419 02:45:53.790063 140363059820416 interactiveshell.py:2882] Step 0 | Loss: 1.3855 | Spent: 24.6 secs | LR: 0.003000
I0419 02:45:55.924041 140363059820416 interactiveshell.py:2882] Step 50 | Loss: 1.3857 | Spent: 2.1 secs | LR: 0.002994
I0419 02:45:58.058234 140363059820416 interactiveshell.py:2882] Step 100 | Loss: 1.3845 | Spent: 2.1 secs | LR: 0.002988
I0419 02:46:00.185530 140363059820416 interactiveshell.py:2882] Step 150 | Loss: 1.3740 | Spent: 2.1 secs | LR: 0.002982
I0419 02:46:02.306592 140363059820416 interactiveshell.py:2882] Step 200 | Loss: 1.3110 | Spent: 2.1 secs | LR: 0.002976
I0419 02:46:04.441041 140363059820416 interactiveshell.py:2882] Step 250 | Loss: 1.2251 | Spent: 2.1 secs | LR: 0.002970
I0419 02:46:06.568732 140363059820416 interactiveshell.py:2882] Step 300 | Loss: 1.1427 | Spent: 2.1 secs | LR: 0.002963
I0419 02:46:08.724704 140363059820416 interactiveshell.py:2882] Step 350 | Loss: 1.0426 | Spent: 2.2 secs | LR: 0.002957
I0419 02:46:10.869930 140363059820

Reading ../data/wn18/test.txt


I0419 02:46:43.659612 140363059820416 interactiveshell.py:2882] MRR: 0.442| Hits@10: 0.702 | Hits@3: 0.519 | Hits@1: 0.313
I0419 02:46:43.661445 140363059820416 interactiveshell.py:2882] Best MRR: 0.442


...

I0419 03:08:22.219443 140363059820416 interactiveshell.py:2882] MRR: 0.798| Hits@10: 0.949 | Hits@3: 0.926 | Hits@1: 0.668
I0419 03:08:22.224124 140363059820416 interactiveshell.py:2882] Best MRR: 0.810


Reading ../data/wn18/train.txt


I0419 03:08:49.069505 140363059820416 interactiveshell.py:2882] Step 19950 | Loss: 0.0002 | Spent: 29.2 secs | LR: 0.001329
I0419 03:08:51.235039 140363059820416 interactiveshell.py:2882] Step 20000 | Loss: 0.0002 | Spent: 2.2 secs | LR: 0.001326
I0419 03:08:53.394015 140363059820416 interactiveshell.py:2882] Step 20050 | Loss: 0.0002 | Spent: 2.2 secs | LR: 0.001323
I0419 03:08:55.569463 140363059820416 interactiveshell.py:2882] Step 20100 | Loss: 0.0002 | Spent: 2.2 secs | LR: 0.001321
I0419 03:08:57.789127 140363059820416 interactiveshell.py:2882] Step 20150 | Loss: 0.0002 | Spent: 2.2 secs | LR: 0.001318
I0419 03:08:59.960206 140363059820416 interactiveshell.py:2882] Step 20200 | Loss: 0.0002 | Spent: 2.2 secs | LR: 0.001315
I0419 03:09:02.139737 140363059820416 interactiveshell.py:2882] Step 20250 | Loss: 0.0002 | Spent: 2.2 secs | LR: 0.001313
I0419 03:09:04.325982 140363059820416 interactiveshell.py:2882] Step 20300 | Loss: 0.0002 | Spent: 2.2 secs | LR: 0.001310
I0419 03:09:06.

Reading ../data/wn18/test.txt


I0419 03:09:38.220379 140363059820416 interactiveshell.py:2882] MRR: 0.795| Hits@10: 0.947 | Hits@3: 0.923 | Hits@1: 0.665
I0419 03:09:38.225623 140363059820416 interactiveshell.py:2882] Best MRR: 0.810
I0419 03:09:38.226562 140363059820416 interactiveshell.py:2882] MRR not improved over 3 epochs, Early Stop
